<a href="https://colab.research.google.com/github/yashrajkakkad/ml-spotify/blob/master/SVM_GridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/My Drive/ml-spotify/data.csv')
df.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920


In [ ]:
df.describe()

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,year
count,174389.000000,174389.000000,1.743890e+05,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000
mean,0.499228,0.536758,2.328100e+05,0.482721,0.068135,0.197252,5.205305,0.211123,-11.750865,0.702384,25.693381,0.105729,117.006500,0.524533,1977.061764
std,0.379936,0.176025,1.483958e+05,0.272685,0.251978,0.334574,3.518292,0.180493,5.691591,0.457211,21.872740,0.182260,30.254178,0.264477,26.907950
min,0.000000,0.000000,4.937000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1920.000000
25%,0.087700,0.414000,1.661330e+05,0.249000,0.000000,0.000000,2.000000,0.099200,-14.908000,0.000000,1.000000,0.035200,93.931000,0.311000,1955.000000
50%,0.517000,0.548000,2.057870e+05,0.465000,0.000000,0.000524,5.000000,0.138000,-10.836000,1.000000,25.000000,0.045500,115.816000,0.536000,1977.000000
75%,0.895000,0.669000,2.657200e+05,0.711000,0.000000,0.252000,8.000000,0.270000,-7.499000,1.000000,42.000000,0.076300,135.011000,0.743000,1999.000000
max,0.996000,0.988000,5.338302e+06,1.000000,1.000000,1.000000,11.000000,1.000000,3.855000,1.000000,100.000000,0.971000,243.507000,1.000000,2021.000000


In [ ]:
df["key"] = df["key"].astype("category")
df = pd.get_dummies(df, columns=["key"])
df.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920,0,0,0,0,0,1,0,0,0,0,0,0
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920,0,0,0,0,0,1,0,0,0,0,0,0
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920,1,0,0,0,0,0,0,0,0,0,0,0
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920,0,0,1,0,0,0,0,0,0,0,0,0
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
df["mode"] = df["mode"].astype("category")
df = pd.get_dummies(df, columns=["mode"])
df.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,liveness,loudness,name,popularity,release_date,speechiness,tempo,valence,year,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,mode_0,mode_1
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,0.3790,-12.628,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920,0,0,0,0,0,1,0,0,0,0,0,0,1,0
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,0.0809,-7.261,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920,0,0,0,0,0,1,0,0,0,0,0,0,1,0
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0.5190,-12.098,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,0.1280,-7.311,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,0.4020,-6.036,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920,0,0,0,0,0,0,0,0,0,0,1,0,1,0


In [ ]:
len(df.columns), df.columns

(31, Index(['acousticness', 'artists', 'danceability', 'duration_ms', 'energy',
        'explicit', 'id', 'instrumentalness', 'liveness', 'loudness', 'name',
        'popularity', 'release_date', 'speechiness', 'tempo', 'valence', 'year',
        'key_0', 'key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7',
        'key_8', 'key_9', 'key_10', 'key_11', 'mode_0', 'mode_1'],
       dtype='object'))

In [ ]:
mean_popularity = 42
y_prime = [1 if i >= mean_popularity else 0 for i in df["popularity"]]
# y = pd.concat([df.id, pd.Series(y_prime, name="popularity")], axis=1)
y_popl = pd.concat([pd.Series(y_prime, name="popularity")], axis=1)
# y_popl["popularity"].value_counts()
y = y_popl["popularity"]

In [ ]:
drop_cols = ["id", "release_date", "popularity", "artists", "name"]
X = df.drop(drop_cols, axis=1)
len(X.columns), X.columns

(26,
 Index(['acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',
        'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
        'valence', 'year', 'key_0', 'key_1', 'key_2', 'key_3', 'key_4', 'key_5',
        'key_6', 'key_7', 'key_8', 'key_9', 'key_10', 'key_11', 'mode_0',
        'mode_1'],
       dtype='object'))

In [ ]:
# drop_features = ["acousticness", "explicit"]
# X = X.drop(drop_features, axis=1)
# len(X.columns), X.columns

In [ ]:
df.describe()

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,liveness,loudness,popularity,speechiness,tempo,valence,year,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,mode_0,mode_1
count,174389.000000,174389.000000,1.743890e+05,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.00000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000,174389.000000
mean,0.499228,0.536758,2.328100e+05,0.482721,0.068135,0.197252,0.211123,-11.750865,25.693381,0.105729,117.006500,0.524533,1977.061764,0.125966,0.077769,0.108470,0.041746,0.076421,0.09488,0.052905,0.122502,0.061202,0.103843,0.071140,0.063158,0.297616,0.702384
std,0.379936,0.176025,1.483958e+05,0.272685,0.251978,0.334574,0.180493,5.691591,21.872740,0.182260,30.254178,0.264477,26.907950,0.331811,0.267808,0.310974,0.200008,0.265671,0.29305,0.223844,0.327866,0.239702,0.305057,0.257059,0.243247,0.457211,0.457211
min,0.000000,0.000000,4.937000e+03,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,1920.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.087700,0.414000,1.661330e+05,0.249000,0.000000,0.000000,0.099200,-14.908000,1.000000,0.035200,93.931000,0.311000,1955.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.517000,0.548000,2.057870e+05,0.465000,0.000000,0.000524,0.138000,-10.836000,25.000000,0.045500,115.816000,0.536000,1977.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.895000,0.669000,2.657200e+05,0.711000,0.000000,0.252000,0.270000,-7.499000,42.000000,0.076300,135.011000,0.743000,1999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,0.996000,0.988000,5.338302e+06,1.000000,1.000000,1.000000,1.000000,3.855000,100.000000,0.971000,243.507000,1.000000,2021.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
X.shape, y.shape

((174389, 26), (174389,))

In [ ]:
scaler = StandardScaler().fit(X)
X = scaler.transform(X)
X.shape

(174389, 26)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

In [ ]:
%%time
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'],
                     'random_state': [42],
                     'gamma': [1e-4, 1e-3, 1e-2, 1e-1, 1e0],
                     'C': [0.1, 1, 10, 100, 1000]
                     }]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train[:10000], y_train[:10000])

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_true, y_pred))
    print()

# svm = LinearSVC(random_state=42, tol=1e-5, max_iter=10000)
# svm.fit(X_train[:50000], y_train[:50000])
# # print("Train accuracy of svm algo:", svm.score(X_train, y_train))
# # print("Test accuracy of svm algo:", svm.score(X_test, y_test))


# cv_scores = cross_val_score(svm, X_train[:100000], y_train[:100000], cv=3)
# print("Cross Validation scores - ", cv_scores)

# y_train_pred = cross_val_predict(svm, X_train[:100000], y_train[:100000])

# confusion_matrix = confusion_matrix(y_train[:100000], y_train_pred)
# print("Confusion matrix\n", confusion_matrix)

# p_score = precision_score(y_train[:100000], y_train_pred)
# rc_score = recall_score(y_train[:100000], y_train_pred)
# f1_score = f1_score(y_train[:100000], y_train_pred)
# p_score, rc_score, f1_score
# print("Recall score = ", rc_score)
# print("Precision score = ", p_score)
# print("F1 score = ", f1_score)

# print("Classification report\n")
# print(classification_report(y_train[:100000], y_train_pred))

# Tuning hyper-parameters for precision



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Pr

Best parameters set found on development set:

{'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf', 'random_state': 42}

Grid scores on development set:

0.368 (+/-0.000) for {'C': 0.1, 'gamma': 0.0001, 'kernel': 'rbf', 'random_state': 42}
0.368 (+/-0.000) for {'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf', 'random_state': 42}
0.723 (+/-0.029) for {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf', 'random_state': 42}
0.746 (+/-0.043) for {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf', 'random_state': 42}
0.368 (+/-0.000) for {'C': 0.1, 'gamma': 1.0, 'kernel': 'rbf', 'random_state': 42}
0.368 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf', 'random_state': 42}
0.724 (+/-0.030) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf', 'random_state': 42}
0.726 (+/-0.030) for {'C': 1, 'gamma': 0.01, 'kernel': 'rbf', 'random_state': 42}
0.725 (+/-0.024) for {'C': 1, 'gamma': 0.1, 'kernel': 'rbf', 'random_state': 42}
0.695 (+/-0.025) for {'C': 1, 'gamma': 1.0, 'kernel': 'rbf', 'random_state': 42}
0.723 (+/-0.031) for {'C'

In [ ]:
set(y_true)

{0, 1}